In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
from numpy.fft import fft
import detecta
def compute_peaks(data):
    X=fft(data)
    N=len(X)
    n=np.arange(N)
    sr=1/50
    T=N/sr
    freq=n/T
    c=20

    n_oneside=N//2
    x=freq[1:n_oneside]
    y=np.abs(X[1:n_oneside])

    max_peak_height=np.amax(y)/c
    peaks=[]
    while len(peaks)<5:
        peaks=detecta.detect_peaks(y, mph=max_peak_height)
        c+=5
        max_peak_height=np.amax(y)/c
    peaks_x=peaks/T
    peaks_y=y[peaks]

    return peaks_x[0:5], peaks_y[0:5]

In [3]:
def find_fft_points(data, name):
    (indices_peaks, peaks) = compute_peaks(data)
    columns_x=[name + "X#1", name + "X#2", name + "X#3", name + "X#4", name + "X#5"]
    columns_y=[name + "P#1", name + "P#2", name + "P#3", name + "P#4", name + "P#5"]
    x_p = pd.DataFrame(data=indices_peaks).T
    x_p.columns = columns_x
    y_p = pd.DataFrame(data=peaks).T
    y_p.columns = columns_y
    tot_p = pd.concat([x_p, y_p], axis=1)
    return tot_p

In [4]:
def compute_time_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_time_features(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [5]:
def find_time_features(data, name):
    columns = [name + "_mean", name + "_std", name + "_range", name + "_IRQ"]
    properties = [np.mean(data), np.std(data), np.max(data) - np.min(data),
                  np.quantile(data, 0.75) - np.quantile(data, 0.25)]
    d = pd.DataFrame(data=properties).T
    d.columns = columns
    return d

In [6]:
from scipy.signal import savgol_filter

def noise_filter(dataframe):
    for column in dataframe.columns:
        dataframe[column] = savgol_filter(dataframe[column], 10, 2, axis=0)
    return dataframe

In [7]:
def compute_freq_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_fft_points(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [8]:
def collapse(df):
    time_df=compute_time_features(df)
    freq_df=compute_freq_features(df)
    return pd.concat([time_df, freq_df], axis=1)

In [9]:
def transform_to_timeseries(dataframe, class_name, sample_number):
    i = dataframe.shape[0]//sample_number
    j=0
    filtered_df=noise_filter(dataframe)
    df_time_series=pd.DataFrame()
    for count in range(1,i):
        samples_df=filtered_df.iloc[j:sample_number*count, :]
        new_df=collapse(samples_df)
        if(count==1):
            df_time_series=new_df
        else:
            df_time_series = pd.concat([df_time_series, new_df], axis=0)
        j=sample_number*count
    df_time_series['class']=class_name
    return df_time_series

In [10]:
import struct

def recv_msg(sock):
    # Read message length and unpack it into an integer
    msglen = int(sock.recv(8))
    if not msglen:
        return None
    # Read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data

In [16]:
import socket, sys, pandas as pd
HOST = ''
PORT = 1060
ADDR =(HOST,PORT)
def server():
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    s.bind(ADDR)
    s.listen(1)
    print ('the server is listening at',s.getsockname())
    ss,sockname = s.accept()
    i=0
    #accept the complete message
    msg=recv_msg(ss).decode()
    samples=msg.split("|")
    if len(samples) == 6:
        lin_acc_x=list(map(float, samples[0][1:len(samples[0])-1].split(", ")))
        lin_acc_y=list(map(float, samples[1][1:len(samples[1])-1].split(", ")))
        lin_acc_z=list(map(float, samples[2][1:len(samples[2])-1].split(", ")))
        rot_x=list(map(float, samples[3][1:len(samples[3])-1].split(", ")))
        rot_y=list(map(float, samples[4][1:len(samples[4])-1].split(", ")))
        rot_z=list(map(float, samples[5][1:len(samples[5])-1].split(", ")))
        df_len=min([len(lin_acc_x), len(lin_acc_y), len(lin_acc_z), len(rot_x), len(rot_y), len(rot_z)])

        df=pd.DataFrame(columns=["rotationRate.x", "rotationRate.y", "rotationRate.z", "userAcceleration.x", "userAcceleration.y", "userAcceleration.z", "class"])
        df["rotationRate.x"]=rot_x[100:df_len-100]
        df["rotationRate.x"]=df["rotationRate.x"]*2
        df["rotationRate.y"]=rot_y[100:df_len-100]
        df["rotationRate.y"]=df["rotationRate.y"]*2
        df["rotationRate.z"]=rot_z[100:df_len-100]
        df["rotationRate.z"]=df["rotationRate.z"]*2
        df["userAcceleration.x"]=lin_acc_x[100:df_len-100]
        df["userAcceleration.x"]=df["userAcceleration.x"]/5
        df["userAcceleration.y"]=lin_acc_y[100:df_len-100]
        df["userAcceleration.y"]=df["userAcceleration.y"]/5
        df["userAcceleration.z"]=lin_acc_z[100:df_len-100]
        df["userAcceleration.z"]=df["userAcceleration.z"]/5
        df["class"]="upstairs"
        df.to_csv("honor20readings/dataset_honor20_upstairs.csv", index=False)
        #df.to_csv("honor20readings/dataset_honor20_walking.csv", mode="a", index=False, header=False)
    ss.close()

server()


the server is listening at ('0.0.0.0', 1060)
31055
